In [1]:
import tensorflow as tf
import os
import zipfile

In [6]:
local_zip = '/happy-or-sad.zip'
zip_ref = zipfile.ZipFile("happy-or-sad.zip", 'r')
zip_ref.extractall("h-or-s")
zip_ref.close()

In [7]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.999):
            print("\n¡Se alcanzó un 99.9% de accuracy!")
            self.model.stop_training = True

In [8]:
callbacks = myCallback()

In [9]:
#Modelo
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

W0701 20:10:22.387256 15428 deprecation.py:506] From C:\Users\usuario\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

W0701 20:11:26.706127 15428 deprecation.py:323] From C:\Users\usuario\Anaconda3\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        'happy-or-sad/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 80 images belonging to 2 classes.


In [12]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1, 
      callbacks=[callbacks])

Epoch 1/15
8/8 [==============================] - 16s 2s/step - loss: 5.5100 - acc: 0.5859
Epoch 2/15
8/8 [==============================] - 3s 350ms/step - loss: 0.4104 - acc: 0.8172
Epoch 3/15
8/8 [==============================] - 3s 359ms/step - loss: 0.0906 - acc: 0.9719
Epoch 4/15
8/8 [==============================] - 3s 352ms/step - loss: 0.0462 - acc: 0.9812
Epoch 5/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0171 - acc: 1.0000
¡Se alcanzó un 99.9% de accuracy!
8/8 [==============================] - 3s 354ms/step - loss: 0.0159 - acc: 1.0000
